A notebook that allows manual interactions with an RTSP server.  
Intended to increase one's understanding of the RTSP protocol.

In [1]:
import socket
import hashlib
from urllib.parse import urlparse
from send_rtsp_setup import send_rtsp_setup
from send_rtsp_describe import send_rtsp_describe

In [11]:
# Function to read RTSP URLs from a file
def read_rtsp_urls(filename):
    urls = []
    try:
        with open(filename, 'r') as file:
            urls = [line.strip() for line in file.readlines() if line.strip()]
    except FileNotFoundError:
        print(f"Error: File {filename} not found.")
    return urls

In [16]:
class RTSPClient:
    def __init__(self, url):
        self.parsed_url = urlparse(url)
        self.url = self.remove_credentials_from_url()
        self.server_ip = self.parsed_url.hostname
        self.server_port = self.parsed_url.port if self.parsed_url.port else 554
        self.username = self.parsed_url.username
        self.password = self.parsed_url.password
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.session_id = None
        self.auth_realm = None
        self.auth_nonce = None
        self.auth_required = False
        self.cseq = 5

    # Methods defined in other files.
    send_rtsp_setup = send_rtsp_setup
    send_rtsp_describe = send_rtsp_describe
    
    def connect(self):
        try:
            self.client_socket.connect((self.server_ip, self.server_port))
        except Exception as e:
            print(f"Error connecting to server: {e}")

    def close(self):
        self.client_socket.close()

    def remove_credentials_from_url(self):
        # Reconstruct the URL without username and password
        new_url = self.parsed_url._replace(netloc=self.parsed_url.hostname + (f":{self.parsed_url.port}" if self.parsed_url.port else ""))
        return new_url.geturl()

    def send_rtsp_options(self):
        try:
            # Prepare the RTSP OPTIONS request
            cseq = 1
            options_request = f"OPTIONS {self.remove_credentials_from_url()} RTSP/1.0\r\n"
            options_request += f"CSeq: {cseq}\r\n"
            options_request += "User-Agent: Python RTSP Client\r\n"
            options_request += "\r\n"
            
            # Send the request to the RTSP server
            self.client_socket.send(options_request.encode())
            
            # Receive the response from the server
            response = self.client_socket.recv(4096)
            print("RTSP Server Response (OPTIONS):\n")
            print(response.decode())
            
        except Exception as e:
            print(f"Error: {e}")

    def send_rtsp_command(self, command_type="PLAY"):
        try:
            # Validate the command_type
            if command_type not in ["PLAY", "PAUSE", "TEARDOWN"]:
                print(f"Error: Invalid command_type '{command_type}'. Must be 'PLAY', 'PAUSE', or 'TEARDOWN'.")
                return

            # Prepare the RTSP PLAY request
            self.cseq += 1
            command_request = f"{command_type} {self.url} RTSP/1.0\r\n"
            command_request += f"CSeq: {self.cseq}\r\n"
            command_request += "User-Agent: Python RTSP Client\r\n"
            if self.session_id:
                command_request += f"Session: {self.session_id}\r\n"

            # If authentication is required, add the Authorization header
            if self.auth_required and self.auth_realm and self.auth_nonce and self.username and self.password:
                # Compute the digest response
                ha1 = hashlib.md5(f"{self.username}:{self.auth_realm}:{self.password}".encode()).hexdigest()
                ha2 = hashlib.md5(f"{command_type}:{self.remove_credentials_from_url()}&trackID=1".encode()).hexdigest()
                response_digest = hashlib.md5(f"{ha1}:{self.auth_nonce}:{ha2}".encode()).hexdigest()
                command_request += (
                    f"Authorization: Digest username=\"{self.username}\", realm=\"{self.auth_realm}\", nonce=\"{self.auth_nonce}\", uri=\"{self.url}&trackID=1\", response=\"{response_digest}\"\r\n"
                )
            command_request += "\r\n"

            
            # Send the PLAY request
            self.client_socket.send(command_request.encode())
            
            # Receive the response from the server
            response = self.client_socket.recv(4096)
            response_str = response.decode()
            print("RTSP Server Response (PLAY):\n")
            print(response_str)
            
        except Exception as e:
            print(f"Error: {e}")

In [4]:
rtsp_urls = read_rtsp_urls('rtsp_urls.txt')

In [ ]:
client = RTSPClient(rtsp_urls[1][1:-1])
client.connect()
client.send_rtsp_options()
client.send_rtsp_describe()
client.send_rtsp_setup()

In [ ]:
client.send_rtsp_command("PLAY")

In [ ]:
client.send_rtsp_command("PAUSE")

In [ ]:
client.send_rtsp_command("TEARDOWN")

In [15]:
client.close()